# Model Conversion

**To run this file requires PyTorch LTS 1.8 and TorchVision**

This file converts the pretrained ResNet50 PyTorch model
into either a Caffe model or an ONNX model.

In [1]:
target_model_type = "onnx"
# or convert the model to caffe
# target_model_type = "caffe"

In [2]:
image_path = "data/drum.jpg"
labels_path = "data/imagenet_classes.txt"
expected_label = "drum"

tmp_model_base_name = "tmp/model"

Test image:
![drum](data/drum.jpg)

In [3]:
from pathlib import Path
from PIL import Image
import numpy as np
import torch
import torchvision
from torchvision.models import resnet50

from nart.tools.pytorch import export_onnx, convert_v2
from nart.tools.pytorch.module_utils import convert_mode

In [4]:
torch.__version__, torchvision.__version__

('1.8.1', '0.2.1')

In [5]:
Path(tmp_model_base_name).parent.mkdir(parents=True, exist_ok=True)

In [6]:
# read labels
with open(labels_path) as f:
    labels = list(map(str.strip, f.readlines()))

In [7]:
# prepare the input data
img = Image.open(image_path).resize((256, 256)).crop((16, 16, 240, 240))
input_data = np.array(img).transpose(2, 0, 1).astype(np.float32) / 255.0
input_data -= np.array([0.485, 0.456, 0.406]).reshape((3, 1, 1))
input_data /= np.array([0.229, 0.224, 0.225]).reshape((3, 1, 1))
input_data = input_data.reshape((1, 3, 224, 224))

In [8]:
input_data = torch.from_numpy(input_data)
print(input_data.shape)

torch.Size([1, 3, 224, 224])


In [9]:
# use the pretrained model, download it if it does not exists
model = resnet50(pretrained=True)
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [10]:
# verify the predicted result
output_data = model(input_data)
predict_res = output_data.argmax(axis=1)
predict_labels = [labels[i] for i in predict_res.tolist()]
print(predict_labels)
assert predict_labels[0] == expected_label

['drum']


In [11]:
if target_model_type.lower() == "onnx":
    convert_func = export_onnx
else:
    convert_func = convert_v2

In [12]:
# convert the model
with convert_mode():
    convert_func(
        model,
        [tuple(input_data.shape)],
        filename=tmp_model_base_name,
        log=True,
        verbose=True,
        cloze=False,
    )

====convert: start of pytorch-to-onnx export


/home/chielo/miniconda3/envs/nart/lib/python3.7/site-packages/nart_py-1.2.14.dev0-py3.7.egg/nart/tools/pytorch/module_utils/pytorch1_8/symbolic_opset9.py:876: UserWarning: Pooling with count_include_pad=True is not supported in caffe.
  warnings.warn("Pooling with count_include_pad=True is not supported in caffe.")


graph(%input.1 : Float(1, 3, 224, 224, strides=[150528, 50176, 224, 1], requires_grad=1, device=cpu),
      %fc.weight : Float(1000, 2048, strides=[2048, 1], requires_grad=1, device=cpu),
      %fc.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %498 : Float(64, 3, 7, 7, strides=[147, 49, 7, 1], requires_grad=0, device=cpu),
      %499 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %501 : Float(64, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %502 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %504 : Float(64, 64, 3, 3, strides=[576, 9, 3, 1], requires_grad=0, device=cpu),
      %505 : Float(64, strides=[1], requires_grad=0, device=cpu),
      %507 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %508 : Float(256, strides=[1], requires_grad=0, device=cpu),
      %510 : Float(256, 64, 1, 1, strides=[64, 1, 1, 1], requires_grad=0, device=cpu),
      %511 : Float(256, strides=[1], requ

====convert: end of pytorch-to-onnx export
